In [1]:
import torch
from torch import nn
import numpy as np
import pdb
# set device
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [35]:
with open('data/kyoto_lexicon.csv', 'r', encoding='utf-8') as file:
    for i in range(99):
        print(file.readline())

"日本語","英語",

"102世吉田日厚貫首","the 102nd head priest, Nikko TOSHIDA",

"1月15日：成人祭、新年祭","15th January: Seijin-sai (Adult Festival), the New Year Festival",

"1月3日：家運隆盛、商売繁盛祈願祭","3rd January: Prayer Festival for the prosperity of family fortunes and business",

"1月7日：七種粥神事","7th January: Nanakusa-gayu shinji (a divine service for a rice porridge with seven spring herbs to insure health for the new year)",

"21世紀COEプログラム","The 21st Century Center Of Excellence Program",

"21世紀出陣弁当","21-seiki Shutsujin Bento (21st century ""kick-off"" lunch box)",

"2月15日：初卯祭","15th February: Hatsuu-sai or The Rite of the First Rabbit Calendar Day",

"2代将軍足利義詮","the Second Shogun Yoshiakira ASHIKAGA",

"305世中村日玄貫首","the 305th chief priest Nichigan NAKAMURA",

"32世済範入道親王","the 32nd priestly Imperial Prince Saihan",

"3月17日～21日：春の感謝祭","17th-21st March: Spring Thanksgiving Festival",

"400年遠忌","the 400th anniversary of the death",

"45世 池坊専永宗匠","45th Generation Headmaster Ikenobo Sen'ei",

"49日法要","Buddhist memor